**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install yellowbrick
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from class_utils import ColGrid, sorted_order, show_tree
from sklearn.tree import DecisionTreeClassifier

from yellowbrick.contrib.classifier import DecisionViz
from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer
# revert yellowbrick's invasive changes to matplotlib's
# styling; also suppressing deprecation warnings
import warnings
import yellowbrick

with warnings.catch_warnings(record=True) as w:
    yellowbrick.style.rcmod.set_aesthetic('reset')
    yellowbrick.style.rcmod.reset_orig()

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }

# create a directory for storing any outputs
import os
os.makedirs("data", exist_ok=True)
os.makedirs("output", exist_ok=True)

# create a synthetic dataset
_blobs, _labels = datasets.make_blobs(
    n_samples=600, random_state=3,
    cluster_std=0.75, centers=5
)

_df_blobs = pd.DataFrame(np.hstack([_blobs, _labels.reshape(-1, 1)]),
                         columns=['x', 'y', 'label'])
_df_blobs['y'] *= 100
_df_blobs.to_csv("data/blobs_2d.csv", index=False)

del _blobs
del _labels
del _df_blobs

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
cluster_colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k', 'gray']

def scatter_legend(ax, sc, labels, num_colors, color_array,
                   s, edgecolor):
    handles = []
    
    for i in range(num_colors):
        h = mlines.Line2D([0], [0], ls="", color=color_array[i],
                          ms=s, marker=sc.get_paths()[0],
                          markeredgecolor=edgecolor)
        handles.append(h)

    ax.legend(handles=handles, labels=labels)

def plot_data(
    data, cluster_centres=None, color='b', ax=None,
    cluster_colors=cluster_colors,
    edgecolors='k', labels=None,
    center_color='orange', center_size=200,
    legend=True
):
    if ax is None:
        ax = plt.gca()
        
    if labels is None:
        ax.scatter(data[:, 0], data[:, 1], s=50,
                   color=color, edgecolors=edgecolors)
    else:
        c = np.asarray(cluster_colors)[labels]
        
        sc = ax.scatter(data[:, 0], data[:, 1], s=50,
                        c=c, edgecolors=edgecolors,
                        #cmap=plt.cm.get_cmap('category10', np.max(labels)+1)
                       )
        
        if legend:
            nclusts = np.max(labels)+1
            scatter_legend(ax, sc, ['$c_{}$'.format(i) for i in range(nclusts)],
                           nclusts, cluster_colors, s=6, edgecolor='k')
        
    if not cluster_centres is None:
        ax.scatter(cluster_centres[:, 0],
                   cluster_centres[:, 1],
            s=center_size, c=center_color,
            edgecolors=edgecolors)

    ax.grid(ls='--')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_axisbelow(True)

## k-means v balíčku Scikit-Learn

Teraz keď sme preskúmali princípy na ktorých je založený algoritmus $k$-means, pozrieme sa aj na jeho praktickú implementáciu zo známeho balíčka `scikit-learn`. Okrem ilustrácie ako $k$-means aplikovať na dátovú množinu, sa pozrieme aj na niektoré techniky vizualizácie umožňujúce preskúmať určité vlastnosti zhlukov a dokonca pomôcť zvoliť vhodný počet zhlukov $k$.

### Predspracovanie: nezabudnite normalizovať dáta

Na načítanie a predspracovanie dát použijeme náš štandardný pipeline, ktorý **normalizuje**  (štandardizuje) numerické stĺpce. Keďže je algoritmus $k$-means založený na vzdialenostiach, správne škálovanie je kľúčové a **nemali by sme zabudnúť dáta normalizovať** . Ak bude obor rozsah niektorého stĺpca omnoho väčší než u iných stĺpcov a normalizáciu neaplikujeme, daný stĺpec bude mať na výsledky zhlukovania podstatne väčší vplyv než zvyšné stĺpce. To typicky nie je žiaduce.



In [ ]:
# we load the data from the CSV
df = pd.read_csv("data/blobs_2d.csv")

# all inputs are numeric
categorical_inputs = []
numeric_inputs = list(df.columns[:-1])

# the preprocessing pipeline
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy='constant', fill_value='MISSING'),
        OneHotEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

# the preprocessed data and the classes
X = input_preproc.fit_transform(df[categorical_inputs+numeric_inputs])
labels = df["label"]

# let's also keep the unnormalized data
X_unnorm = df[categorical_inputs+numeric_inputs].values

# plot the data
plt.figure(figsize=(6, 5))
plot_data(X)

### Zhlukovanie

Keď už máme pripravené dáta, aplikovať zhlukovanie je veľmi ľahké. Jediné, čo treba spraviť, je vytvoriť objekt triedy `KMeans` z balíčka `scikit-learn` a špecifikovať počet zhlukov na $k=5$. Následne model naladíme na dáta pomocou štandardného rozhrania `fit`. Všimnite si, že teraz realizujeme nekontrolované učenie, takže nepracujeme s vektorom požadovaných výstupov `y`. Metóda  `predict` navracia identifikátory zhlukov.

Keď sme vykonali zhlukovanie, znovu si zobrazíme dátovú množinu: tento raz zafarbíme body podľa vypočítaných identifikátorov zhlukov. To nám umožní overiť či zhlukovanie prebehlo správne.



In [ ]:
model = KMeans(n_clusters=5)
model.fit(X)
clusts = model.predict(X)

plt.figure(figsize=(6, 5))
plot_data(X, labels=clusts, legend=True)

#### Zhlukovanie s nenormalizovanými dátami

Aby sme sa presvedčili prečo je normalizácia dát taká potrebná, aplikujeme si teraz zhklukovanie aj na nenormalizovanú verziu dátovej množiny.



In [ ]:
model = KMeans(n_clusters=5)
model.fit(X_unnorm)
clusts = model.predict(X_unnorm)

plt.figure(figsize=(6, 5))
plot_data(X_unnorm, labels=clusts)

Ako vidno, niektoré zhluky tento raz neboli identifikované celkom správne. Deje sa to preto, lebo rozmer $y$ má teraz omnoho väčší rozsah a prikladá sa mu preto väčšia váha.

### Určenie počtu zhlukov $k$

Vo vyššie uvedenom príklade sme predpokladali, že správny počet zhlukov $k$ poznáme. V praxi to je pravda málokedy: ibaže by sme vopred presne vedeli, čo hľadáme. Ako teda určiť dobrú hodnotu $k$? V skutočnosti na to existuje hneď niekoľko metód.

#### Elbow graf

Jedným zo spôsobov ako určiť dobrú hodnotu $k$ je použiť elbow graf (lakťový graf). Jeho myšlienka spočíva v tom, že sa $k$-means spustí viackrát s rôznymi hodnotami $k$, vypočíta sa pre každý prípad skreslenie (distortion score) a výsledky sa vizualizujú. V grafe potom hľadáme "lakeť" (elbow), t.j. bod s maximálnym zakrivením – kde sa najprudšie mení strmosť grafu. Na vytvorenie grafu použijeme balíček `yellowbrick`, ktorý vie elbow bod nájsť a vizualizovať automaticky pomocou knee point detection algoritmu [[yellowbrick]](#yellowbrick).

Skreslenie sa počíta ako súčet kvadratických chýb (sum of squared errors; SSE), t.j. súčet euklidovských vzdialeností medzi bodmi a zodpovedajúcimi stredmi zhlukov [[yellowbrick](#yellowbrick), [k_research](#k_research)]:
$$
J(C) = \sum*{j=1}^{k} \sum* {x_i \in c_j} | x_i - \mu_j |^2.
$$

Všimnite si, že ide o to isté kritérium, ktoré sa snaží minimalizovať algoritmus $k$-means. Tiež vezmite do úvahy, že nie je možné jednoducho vybrať také $k$ , ktoré bude skreslenie minimalizovať: to by znamenalo vytvoriť toľko zhlukov, koľko je bodov v dátovej množine: potom by sa skreslenie znížilo na nulu, ale výsledkom by nebolo kvalitné zhlukovanie. Intuícia za výberom elbow bodu spočíva v tom, že keď dosiahneme správnu hodnotu $k$, skreslenie by sa malo prudko znížiť, lebo relatívne blízko ku každému bodu by mal existovať nejaký stred zhluku. Pridávanie ďalších zhlukov by však uť nemalo mať veľký efekt: bude len deliť už dobre definované zhluky na menšie časti.

Elbow graf sa dá vytvoriť aj pomocou iných kritérií, napr. pomocou Calinski-Harabasz indexu alebo pomocou Silhoutte skóre  [[yellowbrick](#yellowbrick), [ch_index](#ch_index)]: pokojne experimentujte aj s týmito možnosťami. Silhoutte skóre budeme ešte používať aj v neskoršej časti notebook-u v rámci Silhoutte analýzy. 

#### Elbow graf: príklad

Použime teraz elbow graf na určenie najlepšieho $k$ pre našu dátovú množinu. Použijeme triedu `KElbowVisualizer` z balíčka `yellowbrick` a vyskúšame $k \in \{2, 3, ..., 9\}$. Keďže už vieme, že korektný počet zhlukov je v našom prípade  5, mali by sme vidieť, že lakeť grafu je na $k=5$.



In [ ]:
visualizer = KElbowVisualizer(model, k=(2, 10), timings=False)
visualizer.fit(X)
visualizer.ax.grid(ls='--')
visualizer.finalize()

plt.savefig("output/kmeans_elbow.svg", bbox_inches='tight', pad_inches=0)

### Silhouette analýza

Silhouette analýza predstavuje ďalší spôsob ako zvoliť vhodnú hodnotu $k$. Poskytuje ale tiež spôsob ako pre každé $k$ vizualizovať niektoré kľúčové vlastnosti zhlukov. Prístup je založený na Silhoutte koeficiente, ktorý je definovaný takto [[k_research]](#k_research):
$$
s(x_i) = \frac{
    b_i - a_i
}{
    \max{ a_i, b_i }
},
$$

kde $a_i$ je **vnútrozhluková rozličnosť**  (intra-cluster dissimilarity), t.j. priemerná vzdialenosť vzorky $x_i$ od všetkých ostatných vzoriek z toho istého zhluku; a $b_i$ je **medzizhluková rozličnosť**  (inter-cluster dissimilarity), t.j. najkratšia 
vzdialenosť ku vzorke z iného zhluku. Čím nižšia je vnútrozhluková rozličnosť $a_i$, tým viac by mal bod $x_i$ patriť do daného zhluku. Čím väčšia je medzizhluková rozličnosť $b_i$, tým menej by mala vzorka $x_i$ patriť do hociktorého iného zhluku [[k_research]](#k_research).

**Silhouette skóre**  je priemer Silhouette koeficientov naprieč všetkými vzorkami $x_i \in X$:
$$
S = \frac{\sum_{x_i \in X} s(x_i)}{|X|},
$$
kde $X$ je dátová množina a $|X|$ je počet vzoriek v nej (jej kardinalita).

Čím vyššie je Silhouette skóre, do tým väčšej miery by mali v priemere body patriť do svojich zhlukov a nie do hociktorých iných zhlukov. Môžeme si preto zobraziť Silhouette skóre pre viacero hodnôt $k$ a zvoliť $k$ s najvyšším skóre. Na vytvorenie grafu znovu použijeme `KElbowVisualizer`, tento raz však špecifikujeme ako metriku `silhouette`. Je zrejmé, že najvyššiu hodnotu nadobúda pre $k=5$.



In [ ]:
visualizer = KElbowVisualizer(model, k=(3, 8),
                              metric='silhouette',
                              timings=False)
visualizer.fit(X)
visualizer.ax.grid(ls='--')
visualizer.finalize()

Na Silhouette analýze je však skvelé, že poskytuje aj spôsob ako vizualizovať kľúčové vlastnosti všetkých jednotlivých zhlukov. Pre všetky body sa vypočítajú Silhouette koeficienty, zoskupia sa podľa zhlukov, zoradia sa podľa veľkosti a vykreslia sa.

Výsledné grafy indikujú aké veľké sú jednotlivé zhluky a s akou mierou istoty by mal každý jednotlivý bod patriť do daného zhluku a nie do iných zhlukov: body z nižšími Silhouette koeficientami sú pravdepodobne na okraji zhluku a body s veľmi nízkymi koeficientmi môžu byť priradené k nesprávnym zhlukom.

V nasledujúcich grafoch zobrazujeme vedľa seba Silhouette grafy a bodové grafy, aby sa dali ľahšie vykonať porovnania. Pamätajte však, že v praxi, pri práci s vysokorozmernými dátovými množinami by ste, samozrejme, mali k dispozícii len Silhouette grafy.



In [ ]:
k_range = range(3, 8)
fig, axes = plt.subplots(len(k_range), 2)

for k, ax in zip(k_range, axes):
    model = KMeans(n_clusters=k)
    
    visualizer = SilhouetteVisualizer(
        model, colors=cluster_colors,
        alpha=1.0, ax=ax[0])
    visualizer.fit(X)
    visualizer.ax.grid(ls='--')
    visualizer.finalize()
    
    clusts = model.predict(X)
    plot_data(X, labels=clusts, ax=ax[1])

fig.set_size_inches([10, len(k_range)*3])
plt.tight_layout()

Všimnite si veľmi nízke Silhouette koeficienty niektorých bodov, ktoré indikujú, že pravdepodobne nie sú priradené do správneho zhluku. Pri $k=5$ sa žiadne body s veľmi nízkymi koeficientmi nevyskytujú, čo indikuje, že ide o omnoho kvalitnejšie zhlukovanie. Všimnite si tiež, že pomocou týchto grafov vieme vizuálne porovnať veľkosti zhlukov.

### Interpretácia zhlukov

Nájsť v rámci dátovej analýzy zhluky je väčšinou tá ľahšia časť úlohy: ťažšie ich býva interpretovať. Sada nástrojov potrebných na interpretáciu zhlukov je však v podstate tá istá, ktorá sa používa pri exploratívnej analýze dát. Identifikátory zhlukov môžeme do dátovej množiny pridať ako ďalší stĺpec a následne preskúmať jeho interakcie s inými stĺpcami, tie, ktoré korelujú, si zobraziť podrobnejšie atď.

#### Husličkové grafy

V našom prípade by mohli byť užitočné husličkové grafy: povedia nám akým rozsahom $x$ a $y$ každý zhluk približne zodpovedá. Aby sme uľahčili porovnanie, prikladáme znovu aj bodový graf.



In [ ]:
model = KMeans(n_clusters=5)
model.fit(X)
clusts = model.predict(X)

# add the cluster identifiers as a new column to the dataset
df['cluster'] = clusts

In [ ]:
df

In [ ]:
g = ColGrid(df, ['cluster'], ['x', 'y'])
fig, axes = g.map_dataframe(sorted_order(sns.violinplot),
                  palette=cluster_colors)
fig.set_size_inches((10, 4))

for ax in axes:
    ax.grid(ls='--')
    ax.set_axisbelow(True)

In [ ]:
plot_data(X_unnorm, labels=clusts)

#### Rozhodovací strom

Ďalšou možnosťou je natrénovať si na dátach malý rozhodovací strom a pozrieť sa, aké pravidlá objavil.



In [ ]:
dtree = DecisionTreeClassifier()
dtree.fit(X_unnorm, clusts)
show_tree(dtree, numeric_inputs, filled=False)

Aby sa dalo ľahšie skontrolovať či strom správne identifikoval hranice zhlukov, môžeme si v našom prípade zobraziť aj rozhodovacie hranice vzhľadom na $x$ a $y$.



In [ ]:
viz = DecisionViz(
    dtree, features=numeric_inputs,
    classes=['$c_{}$'.format(i) for i in range(np.max(clusts)+1)]
)

viz.fit(X_unnorm, clusts)
viz.draw(X_unnorm, clusts)
viz.show()

### References

<a id="k_research">[k_research]</a> Yuan, C. and Yang, H., 2019. Research on K-value selection method of K-means clustering algorithm. J—Multidisciplinary Scientific Journal, 2(2), pp.226-235.

<a id="ch_index">[ch_index]</a> Wang, X. and Xu, Y., 2019, July. An improved index for clustering validation based on Silhouette index and Calinski-Harabasz index. In IOP Conference Series: Materials Science and Engineering (Vol. 569, No. 5, p. 052024). IOP Publishing.

<a id="yellowbrick">[yellowbrick]</a> Yellowbrick. URL: <https://github.com/DistrictDataLabs/yellowbrick>.

